In [ ]:
import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
import pypdf

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import pdfplumber

with pdfplumber.open("documents/progress report-1-10.pdf") as pdf:
    file_contents = ""
    for page in pdf.pages:
        file_contents += page.extract_text()
file_contents

In [ ]:
import getpass
import os

os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(model="embed-english-v3.0")

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_ezfQK_D8tVnyuq3xKxvpYvXEoHhk42VjHVgB9aq7yLFyHdo2sUmVBnQj5hV1DXJLdLHnb")
index = pc.Index("nlp")

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    all_text = ""

    # Iterate over each page
    for page in doc:
        all_text += page.get_text()

    doc.close()
    return all_text

if __name__ == "__main__":
    pdf_path = "documents/progress report-1-10.pdf"  
    text = extract_text_from_pdf(pdf_path)
    print(text)


In [ ]:
import os
from pathlib import Path

def read_markdown_files(folder_path):
    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' does not exist")
        return
    
    # Get all .md files in the folder
    markdown_files = Path(folder_path).glob('*.md')
    
    # Counter for files found
    files_found = 0
    
    # Read and print contents of each markdown file
    for md_file in markdown_files:
        files_found += 1
        print(f"\n{'='*50}")
        print(f"File: {md_file.name}")
        print(f"{'='*50}")
        
        try:
            with open(md_file, 'r', encoding='utf-8') as file:
                content = file.read()
                print(content)
        except Exception as e:
            print(f"Error reading file {md_file.name}: {str(e)}")
    
    if files_found == 0:
        print("No markdown files found in the specified folder")

# Usage
folder_path = r"C:\Users\Dell\OneDrive\Desktop\project\markdownfiles"  # Your folder path
read_markdown_files(folder_path)

In [ ]:
# # Example documents
# documents = ["documents/progress report-1-10.pdf"]

# # Generate embeddings and push to Pinecone
# for i, doc in enumerate(documents):
#     with fitz.open(doc) as fitz_doc:  # Open the PDF file
#         for page_num, page in enumerate(fitz_doc):
#             page_text = page.get_text()  # Extract text from the page
#             embedding = embeddings.embed_documents([page_text])[0]  # Generate embedding
            
#             # Add metadata to the vector
#             metadata = {
#                 "document": doc,
#                 "page": page_num,
#                 "text": page_text
#             }
            
#             # Update Pinecone with embedding and metadata
#             index.upsert(vectors=[
#                 {
#                     "id": f"doc-{i}-page-{page_num}",
#                     "values": embedding,
#                     "metadata": metadata
#                 }
#             ])

In [ ]:
from pathlib import Path

# Get all markdown files from the specified directory
markdown_dir = r"C:\Users\Dell\OneDrive\Desktop\project\markdownfiles"
markdown_files = list(Path(markdown_dir).glob('*.md'))

# Process each markdown file
for i, md_file in enumerate(markdown_files):
    try:
        # Read markdown file content
        with open(md_file, 'r', encoding='utf-8') as file:
            content = file.read()
            
        # Generate embedding for the content
        embedding = embeddings.embed_documents([content])[0]
        
        # Add metadata
        metadata = {
            "document": str(md_file.name),
            "file_path": str(md_file),
            "file_type": "markdown",
            "text": content
        }
        
        # Update Pinecone with embedding and metadata
        index.upsert(vectors=[
            {
                "id": f"md-{i}",
                "values": embedding,
                "metadata": metadata
            }
        ])
        print(f"Successfully embedded and uploaded: {md_file.name}")
        
    except Exception as e:
        print(f"Error processing {md_file.name}: {str(e)}")

In [ ]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama3-70b-8192", model_provider="groq")

In [ ]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing_extensions import List, TypedDict

from langgraph.graph import START, StateGraph


prompt = hub.pull("rlm/rag-prompt")
prompt

In [ ]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()



In [ ]:
response = graph.invoke({"question": "what do you know about acdemic background of Aditya Vinayak "})
print(response["answer"])

In [ ]:
response = graph.invoke({"question": "How much attendance does Aditya Vinayak have in the Fundamentals of DBMS?"})
print(response["answer"])